# Importing Libraries

In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.image import resize

In [2]:
# Loading Model
model = tf.keras.models.load_model('Trained_model.h5')

In [3]:
classes = ['Aedes Aegypti', 'Anopheles Stephensi', 'Culex Pipiens']

## Single Audio Preprocessing

In [4]:
# Load and preprocess the audio file

def load_and_preprocess_file(file_path, target_shape=(180, 180)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)

    chunk_duration = 2
    overlap_duration = 1

    # convert duration to samples
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate

    # calculate the number of chunks
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1

    # iterate over the chunks
    for i in range(num_chunks):
        # calculate the start and end of the chunk
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples

        # extract the chunk
        chunk = audio_data[start:end]

        # mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)

        # resize the mel spectrogram
        mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)

        # compute the mfcc
        mfcc = librosa.feature.mfcc(y=chunk, sr=sample_rate, n_mfcc=13)

        # resize the mfcc
        mfcc = resize(np.expand_dims(mfcc, axis=-1), target_shape)  

        # combined the mel spectrogram and mfcc
        combined_features = np.concatenate([mel_spectrogram, mfcc], axis=-1)

        # Append the mel spectrogram to the data list
        data.append(combined_features)

    return np.array(data)

In [5]:
file_path = '10130f_01.wav'
X_test = load_and_preprocess_file(file_path)

In [6]:
X_test.shape

(267, 180, 180, 2)

### Model Prediction

In [7]:
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred, axis=1)
    unique_elements, counts = np.unique(predicted_categories, return_counts=True)
    max_count = np.max(counts)
    max_elements = unique_elements[counts == max_count]

    return max_elements[0]

In [8]:
c_index = model_prediction(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 375ms/step


In [9]:
c_index

2

In [10]:
print("Mosquito Species :: Model Prediction --> ", classes[c_index])

Mosquito Species :: Model Prediction -->  Culex Pipiens
